# Chapter 1: Drude Model

#### Drude model considers electrons small solid particles traveling through lattices as a stationary array of heavier and bigger atoms. The model use "mean free time" as a measure of the frequency an electron gets scattered. This jupyter notebook makes use of this concept and uses stochastic method to simulat electrons traveling through a 2D material of finite width but infinite length.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from numpy import random
from ipywidgets import interactive
from matplotlib import animation
from IPython.display import HTML
%matplotlib widget

#### First we setup the simulation parameters such as the external electric field, number of electrons (default to 1) as well as the pseudo-time steps to take. Also, we define the various physical constants such as elementary electron charge, (bare) electron mass, and mean free time ($\tau$). For the simplicity of simulation, we also deifine a constant called $V_0$ as an average restarting velocity after a collison.

In [ ]:
e = -1.602E-19; # Electron charge (C)
m = 9.109E-31; # (Bare) electron mass (kg)
tau = 2000; # Mean free time (s)
v0 = 5; # Average restarting velocity (m/s)
steps = 500; # number of updates to simulate

#### Now We setup up four cases for easy switching in between with the first case corresponding to:
    1. Only Electric field.
    2. Only Magnetic field.
    3. Having both electric and magnetic fields.
    4. Duplicate of case 3 as a playground allowing adjustment to the variables.

In [ ]:
# Initial condition
case = 4;

if case == 1:
    num = 1; # number of electrons to simulate
    Ex = -1E-9; # X component of the external electric field
    Ey = 0; # Y component of the external electric field
    Bz = 0; # Z component of the external magnetic field
    yBnd = 2*num*0.001; # physical boundry of the conductor in y direction
elif case == 2:
    num = 1; # number of electrons to simulate
    Ex = 0; # X component of the external electric field
    Ey = 0; # Y component of the external electric field
    Bz = 5E-9; # Z component of the external magnetic field
    yBnd = 2*num*0.001; # physical boundry of the conductor in y direction
elif case == 3:
    num = 3; # number of electrons to simulate
    Ex = -1E-9; # X component of the external electric field
    Ey = 0; # Y component of the external electric field
    Bz = 1E-9; # Z component of the external magnetic field
    yBnd = 2*num*0.001; # physical boundry of the conductor in y direction
elif case == 4:
    num = 5; # number of electrons to simulate
    Ex = -3E-9; # X component of the external electric field
    Ey = 0; # Y component of the external electric field
    Bz = 3E-9; # Z component of the external magnetic field
    yBnd = 2*num*0.001; # physical boundry of the conductor in y direction
else: # Default setting
    num = 3; # number of electrons to simulate
    Ex = -1E-9; # X component of the external electric field
    Ey = 0; # Y component of the external electric field
    Bz = 5E-9; # Z component of the external magnetic field
    yBnd = 2*num*0.001; # physical boundry of the conductor in y direction

#### Next, we initiate some containers to store variables in the simulation

In [ ]:
global x, y, vt, vxt
x = np.zeros(num);
x = np.expand_dims(x,axis=0);
# y = 2*np.arange(num)-num;
y = random.rand(num)*yBnd-yBnd/2;
y = np.expand_dims(y,axis=0);

vx = np.zeros(num);
vy = np.zeros(num);
vt = []; # Instantaneous speed
vxt = []; # Instantaneous velocity in x-direction
colors = []; # Color assignment to trace different electrons
for i in range(num):
    colors.append('#%06X' % random.randint(0, 0xFFFFFF))

#### next we create the figure objects, and the function to call between each frame for simulation of each time step

In [ ]:
# Create figure objects
figs = plt.figure();
ax1 = figs.add_subplot(1,1,1);
# past, = ax1.plot([],[],'o', markersize = 4, zorder = 5);
# past, = ax1.plot([],[],'.', color = 'darkgrey', zorder = 5);
# init, = ax1.plot([],[],'ok', markersize = 7, markerfacecolor = 'black',zorder = 10)
# end, = ax1.plot([],[],'or', markersize = 7, markerfacecolor = 'red',zorder = 10)
init, = ax1.plot([],[],'ok', markersize = 7, zorder = 10)
end, = ax1.plot([],[],'or', markersize = 7, zorder = 10)

# ax1.set_xlim(-0.2, 0.2);
ax1.set_ylim(-1.2*yBnd,1.2*yBnd);
ax1.set_xlabel('Distance(x)');
ax1.set_ylabel('Distance(y)');

figs2 = plt.figure();
ax2 = figs2.add_subplot(2,1,1);
ax2.set_xlabel('Time');
ax2.set_ylabel('Speed');
speed, = ax2.plot([],[],'-k')

ax3 = figs2.add_subplot(2,1,2)
ax3.set_xlabel('Time');
ax3.set_ylabel('Velocity');
xVelo, = ax3.plot([],[],'-r')

plt.show()
# Initialize the frame
def initf():
    init.set_data(x[0,:],y[0,:])
    return init,

# Animation function which updates figure data.  This is called sequentially
def animate(step,Ex,Ey,vx,vy,tau,v0):
    global x,y,vt,vxt
    delta_T = 0.0001; # time step in seconds
    newX = np.zeros(num);
    newY = np.zeros(num);
    newVt = np.zeros(num);
    for nn in range(num):
        vx_old = vx[nn]; # Use a temporary variable to store the previous value
        vy_old = vy[nn]; # Use a temporary variable to store the previous value
        vx[nn] = vx[nn] + e*(Ex - Bz*vy_old)*delta_T/m; # Update the velocity
        vy[nn] = vy[nn] + e*(Ey + Bz*vx_old)*delta_T/m; # Update the velocity
        newX[nn] = x[step-1][nn] + vx[nn]*delta_T; # Update the coordinate
        newY[nn] = y[step-1][nn] + vy[nn]*delta_T; # Update the coordinate
        if(random.rand() < 1/tau):
            theta = random.rand()*2*np.pi; # pick a random angle in 2D
            vec = random.rand()*v0;
            vx[nn] = np.cos(theta)*vec;
            vy[nn] = np.sin(theta)*vec;
        if(newY[nn] > yBnd):
            newY[nn] = yBnd; # Restrict movements of the electrons in y-direction by a hard and elastic physical boundary
            vy[nn] = -np.absolute(vy[nn]);
        elif(newY[nn] < -yBnd):
            newY[nn] = -yBnd;
            vy[nn] = np.absolute(vy[nn]);
        newVt[nn] = np.linalg.norm([vx[nn],vy[nn]]);

    x = np.concatenate((x, newX[None,:]), axis=0); # Append the new coordinate
    y = np.concatenate((y, newY[None,:]), axis=0); # Append the new coordinate
    vt = np.append(vt,np.mean(newVt)); # Append the speed of the current moment
    vxt = np.append(vxt,np.mean(vx)); # Append the x-velocity of the current moment

#     past.set_data(x,y);
    for ii in range(num):
        ax1.plot(x[len(x)-1,ii], y[len(y)-1,ii], '.', color = colors[ii], zorder = 0);

    end.set_data(x[len(x)-1,:],y[len(y)-1,:]);
#     ax1.set_xlim(np.min(x),np.max(x));
    
    speed.set_data([xx*delta_T for xx in range(1,step+1)],vt);
    ax2.set_xlim(0,step*delta_T);
    ax2.set_ylim(0,1.1*np.max(vt),auto=True);
    ax2.legend(['|V|'],loc='upper right');
    
    xVelo.set_data([xx*delta_T for xx in range(1,step+1)],vxt);
    ax3.set_xlim(0,step*delta_T);
    ax3.set_ylim(np.min(vxt),1.1*np.max(vxt),auto=True);
    ax3.legend(['Vx'],loc='upper right')
    
    return past, end, speed, xVelo

# Call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(figs, animate, init_func=initf, fargs = (Ex,Ey,vx,vy,tau,v0), frames=range(1,steps+1), interval=100, blit = True, repeat = False)

#### Lastly, we compute and plot the average speed and average velocity in x direction over the entire simulation to compare with their instantaneous values. In particular the mean velocity in x direction is what's called drift velocity, and ultimately determines the conductivity/resistivity of the material in Drude model

In [ ]:
# Calculate and plot the average x-velocity and speed
ax2.plot([0, steps+1],[np.mean(vt), np.mean(vt)],'-');
ax2.set_ylim(0,np.max(vt));
ax2.legend(['|V|','mean(|V|)'],loc='upper right');

ax3.plot([0, steps+1],[np.mean(vxt), np.mean(vxt)],'-');
ax3.set_ylim(np.min(vxt),np.max(vxt));
ax3.legend(['Vx','mean(Vx)'],loc='upper right');